In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%pip install transformers
%pip install setfit
%pip install pyarrow

from setfit import SetFitModel, SetFitTrainer
import pandas as pd
import pyarrow as pa
import datasets
import pandas as pd
from os import walk, makedirs
from google.colab import files

prepare data

In [ ]:
df = pd.read_csv("drive/MyDrive/fyp/data_ready/all_data.csv").sample(frac=1)
train_text = list(df["text"])
train_class = list(df["class"])


train_dataset = datasets.Dataset(pa.Table.from_arrays([train_text, train_class], ["text","class"]))

prepare model

In [ ]:
model = SetFitModel.from_pretrained("morit/french_xlm_xnli")

trainer = SetFitTrainer(
  model=model,
  train_dataset=train_dataset,
  metric="accuracy",
  batch_size=16,
  num_iterations=4, # The number of text pairs to generate for contrastive learning
  num_epochs=3, # The number of epochs to use for contrastive learning
  column_mapping={"text": "text", "class": "label"} # Map dataset columns to text/label expected by trainer
)

train model

In [ ]:
# train network
trainer.train()

save model

In [ ]:
trainer.model._save_pretrained("/content")

In [ ]:
!zip -r /content/sample_data.zip /content/sample_data
files.download('/content/sample_data.zip')